<a href="https://colab.research.google.com/github/Tensai-Pro/SRI-PracticeTasks/blob/master/Word%20Embeddings/3)%20fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393484 sha256=7b757473180d73c912186e7b31b3341d4b7c9f11d832c396046c586ac2bc17a2
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import pandas as pd
import numpy as np

from gensim.utils import simple_preprocess
from gensim.models import FastText

import nltk;
nltk.download('stopwords');
from nltk.corpus import stopwords;
# from nltk.stem.snowball import SnowballStemmer
import string

import spacy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import fasttext

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Practice Files/train.zip')
df.head(3)

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...


# Предобработка текста

In [ ]:
stopwords = stopwords.words('english')
stopwords.append("'s")

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def prepare_text(text):
  tokens = nlp(text)
  # tokens = [tok.lemma_.lower() for tok in tokens]
  tokens = [tok.lemma_.lower() for tok in tokens if tok.lemma_ not in stopwords and tok.lemma_ not in string.punctuation]
  return ' '.join(tokens)

In [ ]:
df.Description[0]

"Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again."

In [ ]:
prepare_text(df.Description[0])

'reuters short seller wall street dwindling\\band ultra cynic see green'

In [ ]:
df['clean_text'] = df.Description.apply(prepare_text)
df.head()

,Class Index,Title,Description,clean_text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",reuters short seller wall street dwindling\ban...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,"reuters private investment firm carlyle group,..."
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,reuters soar crude price plus worries\about ec...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,reuters authorities halt oil export\flow main ...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",afp tearaway world oil price topple record str...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.clean_text, df['Class Index'],
                                                    shuffle=True, random_state=42)

In [ ]:
print(len(X_train))
X_train.head()

90000


84145    wind guste 30 mile hour ralph wilson stadium b...
60838    indianapolis base drug manufacturer eli lilly ...
99429    new measure fight crime drug expect heart quee...
26258    new york public school child return class year...
12165    ap timeline sexual assault case los angeles la...
Name: clean_text, dtype: object

# Supervised fastText

## Стандартный пакет fastText

In [ ]:
with open('train_data.txt', 'w', encoding='utf-8') as f:
  for i in range(len(X_train)):
    f.write('__label__' + str(y_train.values[i]) + ' ' + X_train.values[i] + '\n')

with open('test_data.txt', 'w', encoding='utf-8') as f:
  for i in range(len(X_test)):
    f.write('__label__' + str(y_test.values[i]) + ' ' + X_test.values[i] + '\n')

In [ ]:
model = fasttext.train_supervised(input='train_data.txt', epoch=50, wordNgrams=2,
                                  lr=0.005)

In [ ]:
result = model.test('test_data.txt')

In [ ]:
print('Test Accuracy:', result)

Test Accuracy: (30000, 0.9055333333333333, 0.9055333333333333)


In [ ]:
model.predict(X_test.values[10])

(('__label__4',), array([0.92115545]))

In [ ]:
y_test.values[10]

4

## Модуль gensim fastText

In [ ]:
clean_text_tokens = X_train.apply(lambda x: x.split(' '))

In [ ]:
model_gensim = FastText(clean_text_tokens, vector_size=100, window=5,
                        min_count=5, workers=4)

In [ ]:
model_gensim.train(corpus_iterable=clean_text_tokens, 
                   total_examples=len(clean_text_tokens), epochs=10)

(17861656, 19583510)

In [ ]:
model_gensim.wv['gastroenteritis']

array([-0.72896016,  0.07606808,  0.36694753, -0.07568449,  0.07119192,
       -1.0664583 ,  0.378626  , -0.6597585 ,  1.0551571 , -0.21986134,
        0.38397348, -0.5202483 , -0.03893563, -0.37454224, -0.30293787,
        0.01861934, -0.32655925, -0.31563473,  0.04078551,  0.11752435,
       -0.05610346, -0.2316435 , -0.4092508 ,  0.7134848 , -0.2200847 ,
       -0.12685187,  1.2535815 ,  0.29173735,  0.73042035,  0.43831882,
        0.3306061 , -0.6142112 ,  0.32873783, -0.039017  ,  0.04482775,
        0.11474589, -0.21255836, -0.26040208, -0.19691506, -1.4328423 ,
       -0.05345409,  1.1338176 ,  1.3801588 ,  0.47705725,  0.75716144,
        0.3886566 ,  0.29731846, -0.38727447,  0.629503  ,  0.21525204,
       -0.6172844 ,  0.49279812, -0.12916063,  0.624795  , -0.44863018,
       -0.48579386,  0.2085447 , -0.28596294,  1.359474  ,  0.4554377 ,
       -0.84833   ,  0.3223625 ,  0.5812951 , -0.31436485,  0.4969337 ,
        0.67428404, -0.99212873, -0.3689961 ,  0.70073247, -0.25

In [ ]:
model_gensim.wv.most_similar('castle')

[('newcastle', 0.7588932514190674),
 ('nestle', 0.6795447468757629),
 ('winchester', 0.6432747840881348),
 ('hustle', 0.6301568746566772),
 ('manchester', 0.622786283493042),
 ('castillo', 0.604910671710968),
 ('rochester', 0.6043378710746765),
 ('chester', 0.5936146378517151),
 ('cast', 0.5867555141448975),
 ('castilla', 0.5698596239089966)]

## Векторизация

In [ ]:
def vectorize(text):
    vectorized_text = np.array([model_gensim.wv[word] for word in text if word in model_gensim.wv])
    if len(vectorized_text) == 0:
        return np.zeros(100)
    return vectorized_text.mean(axis=0)

In [ ]:
X_train_vectorized = np.array([vectorize(sentence.split(' ')) for sentence in X_train])
X_test_vectorized = np.array([vectorize(sentence.split(' ')) for sentence in X_test])

In [ ]:
X_train_vectorized.shape

(90000, 100)

## Классификация

### MultinomialNB

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train_vectorized)
X_test_scaled = scaler.transform(X_test_vectorized)

In [ ]:
clf_nb = MultinomialNB().fit(X_train_scaled, y_train)

In [ ]:
y_pred_nb = clf_nb.predict(X_test_scaled)

In [ ]:
acc_score = clf_nb.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 72.83666666666667%


### KNN

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=15).fit(X_train_scaled, y_train)
y_pred_knn = clf_knn.predict(X_test_scaled)
acc_score = clf_knn.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 87.58%


# Unsupervised fastText

In [ ]:
values = X_train.values
print(len(values))
print(values[:3])

90000
['wind guste 30 mile hour ralph wilson stadium buffalo bills 12 yard jet 39 end zone sunday willis mcgahee take handoff cradle ball pause'
 'indianapolis base drug manufacturer eli lilly eliminate 575 us job order streamline operation become competitive'
 'new measure fight crime drug expect heart queen speech tuesday']


In [ ]:
values = X_train.values
with open('train_data_unsupervised.txt', 'w', encoding='utf-8') as f:
  for i in range(len(X_train)):
    f.write(values[i] + '\n')

In [ ]:
unsupervised_model = fasttext.train_unsupervised('train_data_unsupervised.txt')

In [ ]:
unsupervised_model.words[:10]

['</s>',
 'say',
 '39;s',
 'new',
 'year',
 'reuters',
 'company',
 'two',
 'us',
 'first']

In [ ]:
unsupervised_model.save_model('/content/gdrive/MyDrive/Practice Files/fasttest_unsuper_model.bin')

In [ ]:
unsupervised_model.get_word_vector('new')

array([ 0.41860446,  0.5429493 , -0.150806  ,  0.21164289, -0.11484031,
        0.04691456, -0.2376853 , -0.24291867,  0.35181773,  0.05803493,
       -0.19746974,  0.33210647,  0.01873512,  0.08071205, -0.16062137,
        0.02923411, -0.06773145, -0.15076815,  0.40037006, -0.22786367,
        0.00284972,  0.13263685,  0.06684748, -0.15789425, -0.05485745,
        0.11893925,  0.3933481 ,  0.14423129, -0.40279076,  0.0138737 ,
        0.1411315 ,  0.2955142 , -0.1177451 ,  0.39018166, -0.1394085 ,
        0.49968675,  0.26523826, -0.12493184, -0.16680555,  0.16615224,
        0.2198517 ,  0.21780245,  0.12479654, -0.4615382 , -0.00142735,
       -0.3198429 , -0.3535077 , -0.03248538, -0.00994665, -0.33196074,
       -0.65212524, -0.32932144, -0.48759693, -0.02870524,  0.01618671,
       -0.36995843, -0.5219891 ,  0.63803524, -0.19380373, -0.30119488,
       -0.2904145 , -0.11124222,  0.15213613, -0.05127013, -0.3265048 ,
        0.07221315,  0.05076874,  0.22602995,  0.07239656,  0.21

In [ ]:
unsupervised_model.get_nearest_neighbors('robot')

[(0.8797124624252319, 'robotic'),
 (0.7036039233207703, 'telescope'),
 (0.6822026968002319, 'hubble'),
 (0.6722701191902161, 'rover'),
 (0.6636658310890198, 'jupiter'),
 (0.6599425673484802, 'voyager'),
 (0.6533786058425903, 'orbiter'),
 (0.6456658840179443, 'odyssey'),
 (0.6392567157745361, 'flyer'),
 (0.6370058655738831, 'mars')]

In [ ]:
unsupervised_model.get_nearest_neighbors('enviroment')

[(0.9230517148971558, 'environment'),
 (0.8595098853111267, 'environmentally'),
 (0.8267915844917297, 'environmental'),
 (0.8260120749473572, 'environmentalists'),
 (0.8214402794837952, 'environmentalist'),
 (0.7405422925949097, 'beehive'),
 (0.7328943014144897, 'wwf'),
 (0.6928366422653198, 'ecosystem'),
 (0.6826496124267578, 'autonomic'),
 (0.6825185418128967, 'therefore')]

In [ ]:
unsupervised_model.get_analogies('paris', 'france', 'moscow')

[(0.6149189472198486, 'russian'),
 (0.5823895931243896, 'russias'),
 (0.5645284056663513, 'russia'),
 (0.5602371096611023, 'russians'),
 (0.526540994644165, 'belarussian'),
 (0.5101833939552307, 'u.s.-russian'),
 (0.5070475339889526, 'chechens'),
 (0.5042608976364136, 'borussia'),
 (0.49830353260040283, 'chechen'),
 (0.48981741070747375, 'vladimir')]

In [ ]:
def unsupervised_vectorize(text):
  vectorized_text = np.array([unsupervised_model.get_word_vector(word) for word in text])
  print(vectorized_text)
  print(vectorized_text.shape)

  if len(vectorized_text) == 0:
    return np.zeros(100)

  return vectorized_text.mean(axis=0)

In [ ]:
# unsupervised_vectorize(X_train.iloc[0].split(' '))

In [ ]:
X_train_unsupervised = np.array([unsupervised_vectorize(sentence.split(' ')) for sentence in X_train])
X_test_unsupervised = np.array([unsupervised_vectorize(sentence.split(' ')) for sentence in X_test])

In [ ]:
# print(X_train.iloc[0])
# print(len(X_train.iloc[0].split(' ')))
# print(X_train_unsupervised[0])
# print(len(X_train_unsupervised[0]))

## Классификация

### MultinomialNB

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train_unsupervised)
X_test_scaled = scaler.transform(X_test_unsupervised)

In [ ]:
clf_nb = MultinomialNB().fit(X_train_scaled, y_train)

In [ ]:
y_pred_nb = clf_nb.predict(X_test_scaled)

In [ ]:
acc_score = clf_nb.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 85.49%


### KNN

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=22).fit(X_train_scaled, y_train)

In [ ]:
y_pred_knn = clf_knn.predict(X_test_scaled)

In [ ]:
acc_score = clf_knn.score(X_test_scaled, y_test) * 100
print(f'Accuracy: {acc_score}%')

Accuracy: 90.01333333333334%


In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(KNeighborsClassifier(),
                           {
                              'n_neighbors':np.arange(22, 30),
                            }, 
                           cv=5, scoring="accuracy",verbose=1,n_jobs=-1
                           )

grid_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': array([22, 23, 24, 25, 26, 27, 28, 29])},
             scoring='accuracy', verbose=1)

In [ ]:
grid_search.best_params_

{'n_neighbors': 22}